### CS 421 PROJECT

In this project, you will be working with data extracted from famous recommender systems type datasets: you are provided with a large set of interactions between users (persons)  and items (movies). Whenever a user "interacts" with an item, it watches the movie and gives a mark or "rating" between 1 and 5 stars (5 stars indicating that the user liked that movie very much, and 1 star indicating that the user didn't like the movie at all. 




In this exercise, we will **not** be performing the recommendation task per se. Instead, we will identify *anomalous users*. In the dataset that you are provided with, some of the data was corrupted. Whilst most of the data comes from real life user-item interactions from a famous movie rating website, some "users" are anomalous: they were generated by me according to some undisclosed procedure. 

You are provided with two data frames: the first one ("ratings") contains the interactions provided to you, and the second one ("labels") contains the labels for the users.

As you can see, the three columns in "ratings" correspond to the user ID, the item ID and the rating. Thus, each row of "ratings" contains a single interaction. For instance, if the row "142, 152, 5" is present, this means that the user with ID 142 has given the movie 152 the rating 5 stars.

The dataframe "labels" has two columns. In the first column we have the user ids, whilst the second column contains the labels. A label of 1 indicates that the user is fake (generated by me), whilst a label of 0 denotes a natural user (coming from real life interactions). 

For instance, if the labels matrix contains the line "142, 1", it means that all of the ratings given by the user with id 142 are fake. This means all lines in the dataframe "ratings" which start with the userID 142 correspond to fake interactions. 

#### Evaluation

Your task is to be able to classify unseen instances as either anomalies or non anomalies (guess whether they are real users or if they were generated by me). 

There are **far more** normal users than anomalies in the dataset, which makes this a very heavily **unbalanced dataset**. Thus, accuracy will not be a good measure of performance, since simply predicting that every user is normal will give good accuracy. Thus, we need to use some other evaluation metrics (see lecture notes from week 3). 

THE **EVALUATION METRICS** are:  THE **AUC** (AREA UNDER CURVE), the **PRECISION**, THE **RECALL**, and the **F1 score**. The **main metric** will be the **AREA UNDER CURVE**, and it will by default be used to rank teams. This means your programs should return an **anomaly score** for each user (the higher the score, the more likely the model think the sample is anomalous).  

Every few weeks, we will evaluate the performance of each team (on an *unseen test set* I will provide) in terms of AUC, PRECISION, RECALL and F1 score, and rank the teams by **AUC** and by F1 score to distinguish between ties, where a tie is defined by a difference of less than 0.005 in AUC.  

The difficulty implied by **the generation procedure of the anomalies WILL CHANGE as the project evolves: depending on how well the teams are doing, I will generate easier or harder anomalies**.

The **first round** will take place after recess (week 9): this means that I will **release the next test set on the tuesday of week 9**, and you must hand in your scores before the **WEDNESDAY at NOON (5th of October)**. We will then look at the results together on the thursday.  

We will check everyone's performance in this way every week (once on  week 10, once on week 11 and once on week 12). 

Whilst performance (expressed in terms of AUC and your ranking compared to other teams) at **each of the check points** (weeks 9 to 12 inclusive) is an **important component** of your **final grade**, the **final report** and the detail of the various methods you will have tried will **also** be very **important**. Ideally, to get perfect marks (A+), you should try at least **two supervised methods** and **two unsupervised methods**, as well as be ranked the **best team** in terms of performance.

The performance part of the grading will be based half on performance at weeks 9,10,11 and half on performance at week 12. 

In [1]:
import numpy as np
import pandas as pd

# Load all previous weeks' data as training sets
batch1=np.load("first_batch.npz")
batch2=np.load("second_batch_with_labels.npz")
batch3=np.load("third_batch_with_labels.npz")

In [2]:
X1=pd.DataFrame(batch1["X"], columns=["user", "item", "rating"])
y1=pd.DataFrame(batch1["y"], columns=["user", "label"])
X2=pd.DataFrame(batch2["X"], columns=["user", "item", "rating"])
y2=pd.DataFrame(batch2["y"], columns=["user", "label"])
X3=pd.DataFrame(batch3["X"], columns=["user", "item", "rating"])
y3=pd.DataFrame(batch3["y"], columns=["user", "label"])

In [7]:
def get_features(X: pd.DataFrame):
  features = pd.DataFrame(X.groupby('user')['user'].max())
  # features['mode'] = X.groupby('user')['rating'].apply(lambda x: x.mode())
  features['mean'] = X.groupby('user')['rating'].mean()
  features['median'] = X.groupby('user')['rating'].median()
  features['max'] = X.groupby('user')['rating'].max()
  features['min'] = X.groupby('user')['rating'].min()
  features['mode'] = X.groupby('user')['rating'].apply(lambda x: x.value_counts().index[0])
  return features

X1_features = get_features(X1)
X2_features = get_features(X2)
X3_features = get_features(X3)


In [10]:
from sklearn.model_selection import train_test_split
# Applying logreg and SVM models for binary classification
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn import svm
# Evaluation metrics: AUC, prec, recall, F1
from sklearn.metrics import auc, roc_auc_score, precision_score, recall_score, f1_score



# Instantiate logreg model and fit logreg
logreg=LogisticRegression(solver='lbfgs', max_iter=1000)

# Fit on first, second and third batch data
logreg.fit(X1_features, y1["label"])
logreg.fit(X2_features, y2["label"])
logreg.fit(X3_features, y3["label"])

LogisticRegression(max_iter=1000)

In [12]:
# Load and predict on fourth batch
batch4 = np.load('fourth_batch.npz')
X4 = pd.DataFrame(batch4['X'], columns=["user", "item", "rating"])
X4.nunique()

user      1300
item       990
rating       6
dtype: int64

In [13]:
X4_features = get_features(X4)

y_pred_logreg = logreg.predict(X4_features)
y_pred_proba = logreg.predict_proba(X4_features)[::,1]


# Evaluation
# print("------Logistic regression evaluation metrics------")
# print(f"AUC: {roc_auc_score(y_test, y_pred_proba)}")
# print(f"Precision: {precision_score(y_test, y_pred_logreg)}")
# print(f"Recall: {recall_score(y_test, y_pred_logreg)}")
# print(f"F1: {f1_score(y_test, y_pred_logreg)}")

In [14]:
predictions = pd.DataFrame(y_pred_proba, columns=['y_pred'])
predictions

,y_pred
0,0.120336
1,0.913030
2,0.128706
3,0.104525
4,0.107561
...,...
1295,0.501180
1296,0.443208
1297,0.489201
1298,0.814640


In [16]:
np.savez('W11_predictions_emmanuel.npz', predictions)